In [1]:
from seagul.rl.ars.ars_pipe import ars
from seagul.nn import MLP
import torch
import matplotlib.pyplot as plt
from seagul.mesh import variation_dim
import time
import copy
import gym


import xarray as xr
import numpy as np
import pandas as pd

In [2]:
def identity(rews,obs,acts):
    return rews

def shrinkdim(rews, obs, acts):
    return rews/variation_dim(obs)

def growdim(rews, obs, acts):
    return rews*variation_dim(obs)

def rough_measure(x):
    d = torch.stack([x[t,:] - x[t-1,:] for t in range(1,x.shape[0])])
    return d.std()#$/(torch.abs(d.mean()))

def rmul(rews, obs, acts):
    return rews*rough_measure(obs)

def rdiv(rews, obs, acts):
    return rews/rough_measure(obs)


torch.set_default_dtype(torch.float64)
env_name = "Swimmer-v2"
exp_names = ['standard', 'rmul']
exp_fns = [identity, rmul]#, rdiv]
num_experiments = len(exp_fns)
num_trials=20
num_epochs=100
n_workers = 12; n_delta = 60; n_top = 20

rewards = xr.DataArray(np.zeros((num_experiments, num_trials, num_epochs)),
                    dims = ("Experiment", "Trial", "Epoch"),
                    coords = {"Experiment": exp_names})

post_rewards = xr.DataArray(np.zeros((num_experiments, num_trials, num_epochs)),
                    dims = ("Experiment", "Trial", "Epoch"),
                    coords = {"Experiment": exp_names})

env = gym.make(env_name)
in_size = env.observation_space.shape[0]
out_size = env.action_space.shape[0]
policy = MLP(in_size,out_size,0,0)policy_dict =  {key:[] for key in exp_names}


data = xr.Dataset({
    "rews" : rewards,
    "post_rews" : post_rewards},
    coords = {"Experiment": exp_names},
    attrs = {"policy_dict":policy_dict,
             "exp_fns":exp_fns,
             "env_name":env_name
            })


running build_ext


In [8]:

import time
start = time.time()
for name, fn in zip(exp_names, exp_fns):
    for i in range(num_trials):
        policy, r_hist, lr_hist = ars(env_name, policy, num_epochs, n_workers=n_workers, n_delta=n_delta, n_top=n_top, postprocess=fn)
        print(i, time.time() - start)
        data.policy_dict[name].append(copy.deepcopy(policy))
        data.rews.loc[name,i,:] = lr_hist
        data.post_rews.loc[name,i,:] = r_hist
        #data.loc[name,i,:] = lr_hist
        #plt.plot(lr_hist)


0 156.13627934455872


ValueError: could not broadcast input array from shape (60) into shape (100)

In [6]:
data.rews

<xarray.DataArray 'rews' (Experiment: 2, Trial: 20, Epoch: 100)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * Experiment  (Experiment) <U8 'standard' 'rmul'
Dimensions without coordinates: Trial, Epoch